In [1]:
import pandas as pd
import numpy as np
import os, time

from math import floor, ceil

# Edit-based

In [2]:
# This code is contributed by mohit kumar 29



def winkler_calculation(distance, constant, count):
   return (distance + (count*constant*(1-distance)))

# Function to calculate the Jaro Similarity of two s
def jaro_distance(s1, s2, constant):
    # Check the first 4 similar chars
    count = 0
    for i, j in zip(s1, s2):
        if i == j and count <= 4:
            count += 1
        else:
            break

    # If both string are equal
    if (s1 == s2):
        return 1.0

    # Length of two strings
    len1 = len(s1)
    len2 = len(s2)

    # Maximum distance upto which matching is allowed
    max_dist = floor(max(len1, len2) / 2) - 1

    # Count of matches
    match = 0

    # Hash for matches
    hash_s1 = [0] * len(s1)
    hash_s2 = [0] * len(s2)

    # Traverse through the first
    for i in range(len1):
        # Check if there is any matches
        for j in range(max(0, i - max_dist),
                       min(len2, i + max_dist + 1)):

            # If there is a match
            if (s1[i] == s2[j] and hash_s2[j] == 0):
                hash_s1[i] = 1
                hash_s2[j] = 1
                match += 1
                break

    # If there is no match
    if (match == 0):
        return 0.0

    # Number of transpositions
    t = 0
    point = 0

    # Count number of occurrences
    # where two characters match but
    # there is a third matched character
    # in between the indices
    for i in range(len1):
        if (hash_s1[i]):

            # Find the next matched character
            # in second
            while (hash_s2[point] == 0):
                point += 1


            if (s1[i] != s2[point]):
                t += 1
            point += 1

    t = (t//2)

    # Return the Jaro Similarity
    return winkler_calculation((match/ len1 + match / len2 +
            (match - t) / match)/ 3.0, constant, count)



# def damerau(t1, t2, tokens_1, tokens_2, distance):
#     # print(f'{t2} --- {tokens_2}')
#     # print(f'--- {tokens_2[t2]}')
#     # print(f'{tokens_1[t1]} == {tokens_2[t2-1]} || {tokens_1[t1-1]} == {tokens_2[t2]}')
#     # if (t1 > 1 and t2 > 1) and (tokens_1[t1] == tokens_2[t2-1]) and (tokens_1[t1-1] == tokens_2[t2]):
#     return min(distance[t1, t2], distance[t1-2, t2-2] + 1)

def levenshteinDistanceDP(token1, token2, damerau_mode=False):


    distances = np.zeros((len(token1) + 1, len(token2) + 1))

    for t1 in range(len(token1) + 1):
        distances[t1][0] = t1

    for t2 in range(len(token2) + 1):
        distances[0][t2] = t2

    a = 0
    b = 0
    c = 0

    for t1 in range(1, len(token1) + 1):
        for t2 in range(1, len(token2) + 1):
            if (token1[t1-1] == token2[t2-1]):
                distances[t1][t2] = distances[t1 - 1][t2 - 1]
            else:
                a = distances[t1][t2 - 1]     # Insertion
                b = distances[t1 - 1][t2]     # Deletion
                c = distances[t1 - 1][t2 - 1] # Substitution


                distances[t1][t2] = min(
                    a + 1,
                    b + 1,
                    c + 1,
                )

                if damerau_mode:
                  if (t1 > 1 and t2 > 1) and (token1[t1-2] == token2[t2-1]) and (token1[t1-1] == token2[t2-2]):
                    distances[t1][t2] = min(
                        distances[t1][t2],
                        distances[t1-2, t2-2] + 1
                    )


    return distances[len(token1)][len(token2)]

def jaccard_similarity(string1, string2):
    intersection = len(list(set(string1).intersection(string2)))
    union = (len(string1) + len(string2)) - intersection
    return float(intersection) / union

In [3]:
if __name__ == '__main__':

    # String
    s1 = "PT. M. Sejahtera_Abadi"
    s2 = "PT. Muhammad Sejahtera"

    constant = float(input('Input Jaro Constant: '))

    # Jaro-Winkler Similarity of Two Strings
    start = time.perf_counter()
    similarity = jaro_distance(s1, s2, constant)
    end = time.perf_counter() - start

    print(f'Jaro-Winkler Similarity: {round(similarity*100, 2)}% | Time: {round(end*1000, 1)} ms')


    # Levenshtein Distance
    start = time.perf_counter()
    similarity = levenshteinDistanceDP(s1, s2)
    end = time.perf_counter() - start

    print(f'Levenshtein Distance: {round(similarity*100/max(len(s1), len(s2)), 2)} | Time: {round(end*1000, 1)} ms')

    # Jaccard Similarity
    start = time.perf_counter()
    similarity = jaccard_similarity(s1, s2)
    end = time.perf_counter() - start

    print(f'Jaccard Similarity: {round(similarity*100, 2)}% | Time: {round(end*1000, 1)} ms')

    # Damerau-Levenshtein Similarity
    start = time.perf_counter()
    similarity = levenshteinDistanceDP(s1, s2, True)
    end = time.perf_counter() - start

    print(f'Damerau-Levenshtein: {round(similarity*100/max(len(s1), len(s2)), 2)} | Time: {round(end*1000, 1)} ms')

Input Jaro Constant: 0.2
Jaro-Winkler Similarity: 100.0% | Time: 0.1 ms
Levenshtein Distance: 59.09 | Time: 1.4 ms
Jaccard Similarity: 41.94% | Time: 0.0 ms
Damerau-Levenshtein: 59.09 | Time: 1.8 ms


# Weighted